In [1]:
import pandas as pd
import numpy as np

pd.set_option("display.max_rows", 30)
pd.set_option('display.max_columns', 40)
import warnings
warnings.filterwarnings('ignore')
%config InlineBackend.figure_format = 'svg'

from fbprophet import Prophet

from tqdm import tqdm

import matplotlib.pylab as plt
%matplotlib inline

import os
os.chdir('..')

In [2]:
transactions = pd.read_csv('transactions.csv')
transactions.head()

customer_id tr_datetime  mcc_code  tr_type    amount term_id
0      7391977  0 10:23:26      4814     1030  -2245.92     NaN
1      7391977  1 10:19:29      6011     7010  56147.89     NaN
2      7391977  1 10:20:56      4829     2330 -56147.89     NaN
3      7391977  1 10:39:54      5499     1010  -1392.47     NaN
4      7391977  2 15:33:42      5499     1010   -920.83     NaN

In [3]:
train_transactions = transactions[transactions.amount < 0].copy()
train_transactions['day'] = train_transactions.tr_datetime.apply(lambda dt: dt.split()[0]).astype(int)
train_transactions.head()

customer_id tr_datetime  mcc_code  tr_type    amount term_id  day
0      7391977  0 10:23:26      4814     1030  -2245.92     NaN    0
2      7391977  1 10:20:56      4829     2330 -56147.89     NaN    1
3      7391977  1 10:39:54      5499     1010  -1392.47     NaN    1
4      7391977  2 15:33:42      5499     1010   -920.83     NaN    2
5      7391977  2 15:53:49      5541     1010 -14643.37     NaN    2

456

In [34]:
pd.date_range(start='2016-01-01', end='2018-01-01', freq='d')[:457]

DatetimeIndex(['2016-01-01', '2016-01-02', '2016-01-03', '2016-01-04',
               '2016-01-05', '2016-01-06', '2016-01-07', '2016-01-08',
               '2016-01-09', '2016-01-10',
               ...
               '2017-03-22', '2017-03-23', '2017-03-24', '2017-03-25',
               '2017-03-26', '2017-03-27', '2017-03-28', '2017-03-29',
               '2017-03-30', '2017-03-31'],
              dtype='datetime64[ns]', length=456, freq='D')

In [65]:
fb_base = train_transactions.groupby(['mcc_code',  'day']).sum()['amount'].reset_index()
sample = fb_base[fb_base.mcc_code == 742].set_index('day').amount.apply(abs)

max_day = train_transactions.day.max()
sample = sample.reindex(np.arange(max_day + 1)).fillna(0).reset_index()
sample['ds'] = pd.date_range(start='2016-01-01', end='2018-01-01', freq='d')[:max_day + 1]
sample = sample[['ds', 'amount']]
sample.columns = ['ds', 'y']
m = Prophet()
m.fit(sample)
future = m.make_future_dataframe(periods=30)
predicted = m.predict(future).yhat[-30:]
predicted

Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.


457    34120.481617
458    22001.029401
459    22268.096601
460    22883.161925
461    32600.280747
462    25695.467852
463    26839.077272
           ...     
480    23357.404153
481    23972.469477
482    33689.588299
483    26784.775404
484    27928.384824
485    35572.891687
486    23453.439470
Name: yhat, Length: 30, dtype: float64

In [71]:
xgb_table = pd.read_csv('xgb_baseline2.csv')
xgb_sample = xgb_table[xgb_table.id.apply(lambda x:x[:3])=='742']
xgb_sample['ds'] = xgb_sample.id.apply(lambda x: int(x[-3:]))
xgb_sample.set_index('ds', inplace=True)
xgb_sample.drop('id', 1, inplace=True)
xgb_sample['fb'] = predicted
xgb_sample

volume            fb
ds                            
457   522.179138  34120.481617
458   220.019836  22001.029401
459   549.382080  22268.096601
460   199.861603  22883.161925
461   199.210510  32600.280747
462   189.391235  25695.467852
463    54.177765  26839.077272
464   190.142609  34483.584135
465  2469.742432  22364.131918
466  1099.043701  22631.199118
467  1731.809814  23246.264442
468  2440.766602  32963.383264
469   547.216431  26058.570370
470   466.992065  27202.179789
471    49.502548  34846.686652
472    51.928146  22727.234435
473    47.143806  22994.301636
474   331.764496  23609.366960
475  1199.358398  33326.485781
476    89.345581  26421.672887
477   135.531631  27565.282307
478   257.166840  35209.789170
479   227.378876  23090.336953
480   554.957642  23357.404153
481   512.033997  23972.469477
482   153.410217  33689.588299
483   550.273499  26784.775404
484   223.778793  27928.384824
485   215.515411  35572.891687
486    55.093956  23453.439470

In [54]:
sample

ds         y
0   2016-01-01  12352.54
1   2016-01-02      0.00
2   2016-01-03      0.00
3   2016-01-04      0.00
4   2016-01-05  28298.54
5   2016-01-06  77484.10
6   2016-01-07  13475.49
..         ...       ...
480 2017-04-25      0.00
481 2017-04-26      0.00
482 2017-04-27      0.00
483 2017-04-28      0.00
484 2017-04-29      0.00
485 2017-04-30      0.00
486 2017-05-01      0.00

[487 rows x 2 columns]